In [ ]:
import cv2
import imutils
import easyocr
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import yaml
import logging
from datetime import datetime

# YAML config
try:
    with open(r".\config.yaml", "r") as f:
        config = yaml.safe_load(f)
except Exception as e:
    raise

# Logger
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(name)s - %(funcName)s - %(message)s",
    filename=config["log_dir"] +
    f"{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.log",
    filemode="w"
)
logger = logging.getLogger(__name__)

logger.info("Config file and logger setup completed.")

In [ ]:
def detect_and_read_plate(image_path):
    """
    Detects license plates in an image and returns the recognized text.

    Args:
        image_path (str): The path to the image file.

    Returns:
        str: The recognized license plate text, or None if no plate is found.
    """
    try:
        # Load image
        img = cv2.imread(image_path)
        if img is None:
            logger.error(f"Could not load image from {image_path}")
            return None

        # Preprocessing
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        noise = cv2.bilateralFilter(gray, 15, 20, 20)
        edged = cv2.Canny(noise, 50, 200)

        # Contour detection
        contours = cv2.findContours(
            edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(imutils.grab_contours(contours),
                          # Keep top 10
                          key=cv2.contourArea, reverse=True)[:10]

        plate_location = None

        # Process each contour
        for contour in contours:
            # Approximate contour to a polygon
            # Reduced tolerance, potentially better results.
            approx = cv2.approxPolyDP(contour, 7, True)

            # Check if the contour has 4 vertices
            if len(approx) == 4:
                # Extract the bounding rectangle
                plate_location = approx
                break

        if plate_location is None:
            logger.warning(f"No license plate found in {image_path}")
            return None

        # Read the plate text
        mask = np.zeros(gray.shape, np.uint8)
        cv2.drawContours(mask, [plate_location], 0, 255, -1)
        cv2.bitwise_and(img, img, mask=mask)

        (a, b) = np.where(mask == 255)
        (a1, b1) = (np.min(a), np.min(b))
        (a2, b2) = (np.max(a), np.max(b))
        plate = gray[a1:a2, b1:b2]

        reader = easyocr.Reader(["en"], verbose=False)
        try:
            results = reader.readtext(plate)
            plate_text = results[0][-2]  # Get the text from the result
            # filter out any non-alphabetic chars.
            plate_text = [l for l in plate_text if l.islower() == 0]
            plate_text = "".join(plate_text)

        except Exception as e:
            logger.error(f"Error reading plate text with EasyOCR: {e}")
            return None

        # Display the image with the detected plate
        rec_img = cv2.rectangle(
            img, (plate_location[0][0]), (plate_location[2][0]), (0, 255, 0), 2)
        plt.imshow(cv2.cvtColor(rec_img, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title(plate_text)
        plt.show()

        return plate_text

    except Exception as e:
        logger.error(f"An unexpected error occurred: {e}")
        return None

In [ ]:
plate = "path"
plate_text = detect_and_read_plate(plate)
if plate_text:
    print(f"Plate detected: {plate_text}")
else:
    print("No plate detected.")